In [79]:
# 共通で利用するライブラリ
import glob
import os
import datetime
import pickle
from natsort import natsorted
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline
%load_ext autoreload

# 追加するライブラリ
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ch.07 機械学習モデルを構築する

- 今月の注文データから来月のオーダー数が増加するか減少するか予測するモデルを作成する
- ml_base_data.csv
  - 機械学習用データ

### Knock61: フォルダを生成して機械学習用データを読み込む

- フォルダ構成

```bash
ch07
├── ch07.ipynb
├── source
└── data
    ├── 0_input
    └── 1_output
```

In [80]:
# フォルダ作成

data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "1_output")

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


In [81]:
# データの準備

ml_data_file = "ml_base_data.csv"
ml_data = pd.read_csv(os.path.join(input_dir, ml_data_file))

display(ml_data)


,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0000,0.0000,1147,699,202,841,306,844,303,91,122,112,101,95,107,106,100,108,109,96,34.1101,201904
1,さいたま南店,1.0000,1.0000,1504,916,287,1105,399,1104,400,130,135,147,143,142,137,130,113,140,132,155,35.3377,201904
2,さいたま緑店,1.0000,1.0000,1028,642,181,756,272,756,272,95,91,106,95,102,82,90,93,95,95,84,34.2916,201904
3,さいたま西店,1.0000,0.0000,1184,708,204,852,332,870,314,122,101,110,117,105,112,103,112,96,108,98,34.5765,201904
4,つくば店,1.0000,1.0000,1267,785,209,928,339,936,331,122,119,105,102,128,117,110,107,100,132,125,34.6635,201904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,高津店,1.0000,0.0000,1080,657,202,787,293,741,339,87,87,94,99,100,108,102,105,102,103,93,34.2198,202002
2141,高田馬場店,1.0000,0.0000,1027,640,190,775,252,709,318,105,101,91,87,96,83,87,96,93,100,88,34.8112,202002
2142,鴻巣店,1.0000,1.0000,1074,654,191,791,283,745,329,97,93,102,108,96,100,99,90,105,103,81,33.0906,202002
2143,鶴見店,1.0000,1.0000,1424,888,225,1046,378,985,439,134,116,135,133,131,127,146,124,129,125,124,34.4887,202002


### Knock62: カテゴリカル変数の対応を行う

In [82]:
# ["store_name"]のOne-Hotエンコーディング

category_data = pd.get_dummies(
    ml_data["store_name"],
    prefix="store",
    prefix_sep="_"
)
display(category_data)

,store_あきる野店,store_さいたま南店,store_さいたま緑店,store_さいたま西店,store_つくば店,store_三浦店,store_三鷹店,store_上尾店,store_上野店,store_世田谷店,store_中原店,store_中店,store_中野店,store_久喜店,store_九段店,store_亀有店,store_五反田店,store_代々木店,store_伊勢原店,store_伊勢崎店,store_佐倉店,store_佐野店,store_保土ケ谷店,store_入間店,store_八千代店,store_八潮店,store_八王子店,store_六本木店,store_前橋店,store_北千住店,store_千葉中央店,store_千葉緑店,store_南砂店,store_南足柄店,store_厚木店,store_取手店,store_向島店,store_君津店,store_和光店,store_品川店,store_四街道店,store_国分寺店,store_国立店,store_坂戸店,store_墨田店,store_多摩店,store_大久保店,store_大井店,store_大和店,store_大塚店,store_大宮店,store_大森店,store_大泉店,store_大田店,store_太田店,store_宇都宮店,store_守谷店,store_宮前店,store_富岡店,store_富津店,store_富里店,store_小山店,store_小平店,store_小田原店,store_小金井店,store_川口店,store_川崎多摩店,store_川崎店,store_川越店,store_巣鴨店,store_市原店,store_市川店,store_平塚店,store_幸店,store_府中店,store_座間店,store_志木店,store_恵比寿店,store_愛甲店,store_成城店,store_成田店,store_我孫子店,store_戸塚店,store_戸田店,store_所沢店,store_新宿店,store_新座店,store_日本橋店,store_日立店,store_日野店,store_旭店,store_春日部店,store_昭島店,store_木更津店,store_本庄店,store_本郷店,store_杉並店,store_東中野店,store_東久留米店,store_東大和店,store_東尾久店,store_東村山店,store_東松山店,store_松戸店,store_板橋店,store_柏店,store_栃木店,store_栄店,store_桐生店,store_横浜南店,store_横浜緑店,store_横須賀店,store_武蔵村山店,store_武蔵野店,store_水戸店,store_江戸川店,store_江東店,store_池尻店,store_池袋店,store_泉店,store_流山店,store_浅草店,store_浦和店,store_浦安店,store_海老名店,store_清瀬店,store_渋谷店,store_港北店,store_港南店,store_瀬谷店,store_熊谷店,store_狛江店,store_狭山店,store_王子店,store_町田店,store_白井店,store_目黒店,store_相模原店,store_石神井店,store_磯子店,store_神奈川店,store_神田店,store_福生店,store_秦野店,store_稲城店,store_立川店,store_綾瀬店,store_練馬店,store_羽村店,store_羽生店,store_習志野店,store_自由が丘店,store_船橋店,store_若葉店,store_茂原店,store_茅ヶ崎店,store_草加店,store_荒川店,store_荻窪店,store_葛飾店,store_蒲田店,store_藤沢店,store_行田店,store_西多摩店,store_西東京店,store_西葛西店,store_調布店,store_谷中店,store_赤坂店,store_赤羽店,store_越谷店,store_足利店,store_足柄上店,store_足立店,store_逗子店,store_那珂店,store_都筑店,store_金沢店,store_銀座店,store_鎌倉店,store_青山店,store_青梅店,store_青葉店,store_館林店,store_駒沢店,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店,store_麻生店
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,.

In [83]:
# カテゴリカル変数を1列消す
# 多重共線性の防止

del category_data["store_麻生店"]

del ml_data["year_month"]
del ml_data["store_name"]

# ml_dataにカテゴリカル変数を結合する
ml_data = pd.concat([ml_data, category_data], axis=1)

ml_data.columns

Index(['y_weekday', 'y_weekend', 'order', 'order_fin', 'order_cancel',
       'order_delivery', 'order_takeout', 'order_weekday', 'order_weekend',
       'order_time_11',
       ...
       'store_駒沢店', 'store_駒込店', 'store_高円寺店', 'store_高島平店', 'store_高崎店',
       'store_高座店', 'store_高津店', 'store_高田馬場店', 'store_鴻巣店', 'store_鶴見店'],
      dtype='object', length=215)

### Knock63: 学習データとテストデータを分割する

In [84]:
# weekdayモデルとweekendモデル用に
# 学習データをテストデータに分割する

train_data, test_data = train_test_split(
    ml_data, test_size=0.3, random_state=0
)

print(f"Train: {len(train_data)}件 / Test: {len(test_data)}件\n")

print(f'Weekday Train0: {len(train_data.loc[train_data["y_weekday"] == 0])}件')
print(f'Weekday Train1: {len(train_data.loc[train_data["y_weekday"] == 1])}件')
print(f'Weekday Test0: {len(test_data.loc[test_data["y_weekday"] == 0])}件')
print(f'Weekday Test1: {len(test_data.loc[test_data["y_weekday"] == 1])}件\n')

print(f'Weekend Train0: {len(train_data.loc[train_data["y_weekend"] == 0])}件')
print(f'Weekend Train1: {len(train_data.loc[train_data["y_weekend"] == 1])}件')
print(f'Weekend Test0: {len(test_data.loc[test_data["y_weekend"] == 0])}件')
print(f'Weekend Test1: {len(test_data.loc[test_data["y_weekend"] == 1])}件\n')


Train: 1501件 / Test: 644件

Weekday Train0: 685件
Weekday Train1: 816件
Weekday Test0: 290件
Weekday Test1: 354件

Weekend Train0: 708件
Weekend Train1: 793件
Weekend Test0: 295件
Weekend Test1: 349件



### Knock64: 1つのモデルを構築する

- Weekdayモデル

In [85]:
# 説明変数と目的変数の作成

X_cols = list(train_data.columns)
X_cols.remove("y_weekday")
X_cols.remove("y_weekend")
target_y = "y_weekday"

y_train = train_data[target_y]
X_train = train_data[X_cols]

y_test = test_data[target_y]
X_test = test_data[X_cols]

display(y_train)
display(X_train)

1137   1.0000
971    0.0000
1983   1.0000
268    0.0000
481    1.0000
        ...  
1033   1.0000
1731   1.0000
763    0.0000
835    0.0000
1653   1.0000
Name: y_weekday, Length: 1501, dtype: float64

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,store_あきる野店,store_さいたま南店,store_さいたま緑店,store_さいたま西店,store_つくば店,store_三浦店,store_三鷹店,store_上尾店,store_上野店,store_世田谷店,store_中原店,store_中店,store_中野店,store_久喜店,store_九段店,store_亀有店,store_五反田店,store_代々木店,store_伊勢原店,store_伊勢崎店,store_佐倉店,store_佐野店,store_保土ケ谷店,store_入間店,store_八千代店,store_八潮店,store_八王子店,store_六本木店,store_前橋店,store_北千住店,store_千葉中央店,store_千葉緑店,store_南砂店,store_南足柄店,store_厚木店,store_取手店,store_向島店,store_君津店,store_和光店,store_品川店,store_四街道店,store_国分寺店,store_国立店,store_坂戸店,store_墨田店,store_多摩店,store_大久保店,store_大井店,store_大和店,store_大塚店,store_大宮店,store_大森店,store_大泉店,store_大田店,store_太田店,store_宇都宮店,store_守谷店,store_宮前店,store_富岡店,store_富津店,store_富里店,store_小山店,store_小平店,store_小田原店,store_小金井店,store_川口店,store_川崎多摩店,store_川崎店,store_川越店,store_巣鴨店,store_市原店,store_市川店,store_平塚店,store_幸店,store_府中店,store_座間店,store_志木店,store_恵比寿店,store_愛甲店,store_成城店,store_成田店,store_我孫子店,store_戸塚店,store_戸田店,store_所沢店,store_新宿店,store_新座店,store_日本橋店,store_日立店,store_日野店,store_旭店,store_春日部店,store_昭島店,store_木更津店,store_本庄店,store_本郷店,store_杉並店,store_東中野店,store_東久留米店,store_東大和店,store_東尾久店,store_東村山店,store_東松山店,store_松戸店,store_板橋店,store_柏店,store_栃木店,store_栄店,store_桐生店,store_横浜南店,store_横浜緑店,store_横須賀店,store_武蔵村山店,store_武蔵野店,store_水戸店,store_江戸川店,store_江東店,store_池尻店,store_池袋店,store_泉店,store_流山店,store_浅草店,store_浦和店,store_浦安店,store_海老名店,store_清瀬店,store_渋谷店,store_港北店,store_港南店,store_瀬谷店,store_熊谷店,store_狛江店,store_狭山店,store_王子店,store_町田店,store_白井店,store_目黒店,store_相模原店,store_石神井店,store_磯子店,store_神奈川店,store_神田店,store_福生店,store_秦野店,store_稲城店,store_立川店,store_綾瀬店,store_練馬店,store_羽村店,store_羽生店,store_習志野店,store_自由が丘店,store_船橋店,store_若葉店,store_茂原店,store_茅ヶ崎店,store_草加店,store_荒川店,store_荻窪店,store_葛飾店,store_蒲田店,store_藤沢店,store_行田店,store_西多摩店,store_西東京店,store_西葛西店,store_調布店,store_谷中店,store_赤坂店,store_赤羽店,store_越谷店,store_足利店,store_足柄上店,store_足立店,store_逗子店,store_那珂店,store_都筑店,store_金沢店,store_銀座店,store_鎌倉店,store_青山店,store_青梅店,store_青葉店,store_館林店,store_駒沢店,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店
1137,977,622,168,724,253,685,292,102,88,84,96,79,97,84,86,85,92,84,34.4784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
971,1099,680,195,816,283,779,320,99,102,101,106,104,91,104,106,98,81,107,34.8750,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1983,966,590,172,724,242,671,295,80,95,87,87,91,101,83,94,88,75,85,34.9484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
268,1432,891,286,1085,347,1065,367,139,137,131,138,142,128,124,136,127,123,107,34.3709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [86]:
# 決定木モデルの構築

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=0)

### Knock65: 評価を実施する

In [87]:
# 構築したモデルでの予測

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

y_pred_test

array([0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0.

- 混合行列 Confusion Matrix

||0|1|
|-:|:-:|:-:|
|0|**TN<br>(True Negative)**|FP<br>(False Positive)|
|1|FN<br>(False Negative)|**TP<br>(True Positive)**|

- 正解率（精度）Accuracy
  - 正解数 / 全体件数
  - (TN + TP) / (TN + FP + FN + TP)
- 再現率 Recall
  - 正解したPositive数 / 本当のPositive数
  - TP / (FN + TP)
- 適合率 Precision
  - 正解したPositive数 / Positiveと予測した数 
  - TP / (FP + TP)
- F値 F-score
  - 再現率と適合率の調和平均
  - 2 * 適合率 * 再現率 / (再現率 + 適合率)

In [88]:
# 時間帯別オーダー数の予測結果の精度指標

# accuracy
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

# recall
recall_train = recall_score(y_train, y_pred_train)
recall_test = recall_score(y_test, y_pred_test)

# precision
precision_train = precision_score(y_train, y_pred_train)
precision_test = precision_score(y_test, y_pred_test)

# F-score
f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

# 表示
print(
    f"[正解率] Train: {round(accuracy_train, 2)} Test: {round(accuracy_test, 2)}"
)
print(
    f"[再現率] Train: {round(recall_train, 2)} Test: {round(recall_test, 2)}"
)
print(
    f"[適合率] Train: {round(precision_train, 2)} Test: {round(precision_test, 2)}"
)
print(f"[F値] Train: {round(f1_train, 2)} Test: {round(f1_test, 2)}")


[正解率] Train: 1.0 Test: 0.81
[再現率] Train: 1.0 Test: 0.81
[適合率] Train: 1.0 Test: 0.83
[F値] Train: 1.0 Test: 0.82


In [89]:
# 混合行列の表示

print(confusion_matrix(y_train, y_pred_train))
print(confusion_matrix(y_test, y_pred_test))

[[685   0]
 [  0 816]]
[[233  57]
 [ 67 287]]


In [90]:
# 混合行列データの格納

# train結果
tn_train, fp_train, fn_train, tp_train = confusion_matrix(
    y_train, y_pred_train
).ravel()
# test結果
tn_test, fp_test, fn_test, tp_test = confusion_matrix(
    y_test, y_pred_test
).ravel()

# 表示
print(f"[混合行列] Train: {tn_train}, {fp_train}, {fn_train}, {tp_train}")
print(f"[混合行列] Test: {tn_test}, {fp_test}, {fn_test}, {tp_test}")

[混合行列] Train: 685, 0, 0, 816
[混合行列] Test: 233, 57, 67, 287


In [91]:
# 精度指標のデータフレーム化

score_train = pd.DataFrame({
    "DataCategory": ["train"],
    "accuracy": [accuracy_train],
    "recall": [recall_train],
    "precision_train": [precision_train],
    "f1": [f1_train],
    "tn": [tn_train],
    "fp": [fp_train],
    "fn": [fn_train],
    "tp": [tp_train]
})
score_test = pd.DataFrame({
    "DataCategory": ["test"],
    "accuracy": [accuracy_test],
    "recall": [recall_test],
    "precision_train": [precision_test],
    "f1": [f1_test],
    "tn": [tn_test],
    "fp": [fp_test],
    "fn": [fn_test],
    "tp": [tp_test]
})
score = pd.concat([score_train, score_test], ignore_index=True)

display(score)

,DataCategory,accuracy,recall,precision_train,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8075,0.8107,0.8343,0.8223,233,57,67,287


### Knock66: モデルの重要度を確認する

In [92]:
# モデルの重要度 Top10

importance = pd.DataFrame({
    "cols": X_train.columns,
    "importance": model.feature_importances_
})
importance = importance.sort_values("importance", ascending=False)

display(importance.head(20))

,cols,importance
5,order_weekday,0.3544
6,order_weekend,0.3478
2,order_cancel,0.0321
3,order_delivery,0.0272
8,order_time_12,0.0256
12,order_time_16,0.0225
0,order,0.0182
13,order_time_17,0.0182
17,order_time_21,0.0158
10,order_time_14,0.0150


### Knock67: モデル構築から評価までを関数化する

In [93]:
# モデル構築から評価までの関数

def make_model_and_eval(
    model, X_train, X_test, y_train, y_test
):
    """
    モデル構築から評価までの関数
    """
    # モデル構築
    model.fit(X_train, y_train)
    # 予測
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # accuracy
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)

    # recall
    recall_train = recall_score(y_train, y_pred_train)
    recall_test = recall_score(y_test, y_pred_test)

    # precision
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)

    # F-score
    f1_train = f1_score(y_train, y_pred_train)
    f1_test = f1_score(y_test, y_pred_test)

    # 精度指標のデータフレーム化
    score_train = pd.DataFrame({
        "DataCategory": ["train"],
        "accuracy": [accuracy_train],
        "recall": [recall_train],
        "precision": [precision_train],
        "f1": [f1_train],
        "tn": [tn_train],
        "fp": [fp_train],
        "fn": [fn_train],
        "tp": [tp_train]
    })
    score_test = pd.DataFrame({
        "DataCategory": ["test"],
        "accuracy": [accuracy_test],
        "recall": [recall_test],
        "precision": [precision_test],
        "f1": [f1_test],
        "tn": [tn_test],
        "fp": [fp_test],
        "fn": [fn_test],
        "tp": [tp_test]
    })
    score = pd.concat([score_train, score_test], ignore_index=True)

    # モデルの重要度
    importance = pd.DataFrame({
        "cols": X_train.columns,
        "importance": model.feature_importances_
    })
    importance = importance.sort_values("importance", ascending=False)

    # 説明変数の列名
    cols = pd.DataFrame({"X_cols": X_train.columns})

    # 表示
    display(score)

    return score, importance, model, cols


In [94]:
# 関数を使用した決定木モデルの構築と評価

model = DecisionTreeClassifier(random_state=0)
score, importance, model, cols = make_model_and_eval(
    model, X_train, X_test, y_train, y_test
)


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8075,0.8107,0.8343,0.8223,233,57,67,287


### Knock68: モデルファイルや評価結果を出力する

In [95]:
# 出力フォルダの作成

now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
target_output_dir_name = "result_" + now
target_output_dir = os.path.join(output_dir, target_output_dir_name)
os.makedirs(target_output_dir, exist_ok=True)

print(target_output_dir)

data/1_output/result_20220924112107


In [96]:
# 評価結果とモデルファイルの出力

score_name = "score.csv"
importance_name = "importance.csv"
cols_name = "X_cols.csv"
model_name = "model.pickle"

score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)
model_path = os.path.join(target_output_dir, model_name)

score.to_csv(score_path, index=False)
importance.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)
with open(model_path, mode="wb") as f:
    pickle.dump(model, f, protocol=2)

### Knock69: アルゴリズムを拡張して多角的な評価を実施する

In [97]:
# 複数アルゴリズムでのモデル構築

# モデルの定義
models = {
    "tree": DecisionTreeClassifier(random_state=0),
    "RandomForest": RandomForestClassifier(random_state=0),
    "GradientBoosting": GradientBoostingClassifier(random_state=0)
}

# フォルダの作成
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
target_output_dir_name = "result_" + now
target_output_dir = os.path.join(output_dir, target_output_dir_name)
os.makedirs(target_output_dir, exist_ok=True)
print(target_output_dir)

score_all = []
importance_all = []
for model_name, model in models.items():
    print(model_name)
    # モデル構築・評価
    score, importance, model, cols = make_model_and_eval(
        model, X_train, X_test, y_train, y_test
    )
    score["model_name"] = model_name
    importance["model_name"] = model_name

    # モデル保存（pickle）
    model_name = f"model_{model_name}.pickle"
    model_path = os.path.join(target_output_dir, model_name)
    with open(model_path, mode="wb") as f:
        pickle.dump(model, f, protocol=2)

    score_all.append(score)
    importance_all.append(importance)

# データフレーム化
score_all = pd.concat(score_all, ignore_index=True)
importance_all = pd.concat(importance_all, ignore_index=True)
cols = pd.DataFrame({"X_cols": X_train.columns})

# ファイル保存（csv）
score_name = "score.csv"
importance_name = "importance.csv"
cols_name = "X_cols.csv"

score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)

score_all.to_csv(score_path, index=False)
importance_all.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)


data/1_output/result_20220924112107
tree


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8075,0.8107,0.8343,0.8223,233,57,67,287


RandomForest


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8059,0.7994,0.8398,0.8191,233,57,67,287


GradientBoosting


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,0.8721,0.8958,0.8723,0.8839,685,0,0,816
1,test,0.8276,0.8418,0.8442,0.8430,233,57,67,287


### Knock70: 平日・休日モデルを一度で回せるようにする

In [98]:
# 平日・休日モデルの構築

X_cols = list(train_data.columns)
X_cols.remove("y_weekday")
X_cols.remove("y_weekend")

targets_y = ["y_weekday", "y_weekend"]

# 出力フォルダの作成
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
target_output_dir_name = "result_" + now
target_output_dir = os.path.join(output_dir, target_output_dir_name)
os.makedirs(target_output_dir, exist_ok=True)
print(target_output_dir)

score_all = []
importance_all = []

for target_y in targets_y:
    y_train = train_data[target_y]
    X_train = train_data[X_cols]
    y_test = test_data[target_y]
    X_test = test_data[X_cols]

    # モデルの定義
    models = {
        "tree": DecisionTreeClassifier(random_state=0),
        "RandomForest": RandomForestClassifier(random_state=0),
        "GradientBoosting": GradientBoostingClassifier(random_state=0)
    }

    for model_name, model in models.items():
        # モデル構築・評価
        print(target_y, model_name)
        score, importance, model, cols = make_model_and_eval(
            model, X_train, X_test, y_train, y_test
        )
        score["model_name"] = model_name
        importance["model_name"] = model_name
        score["model_target"] = target_y
        importance["model_target"] = target_y

        # モデル保存（pickle）
        model_name = f"model_{target_y}_{model_name}.pickle"
        model_path = os.path.join(target_output_dir, model_name)
        with open(model_path, mode="wb") as f:
            pickle.dump(model, f, protocol=2)

        score_all.append(score)
        importance_all.append(importance)

# データフレーム化
score_all = pd.concat(score_all, ignore_index=True)
importance_all = pd.concat(importance_all, ignore_index=True)
cols = pd.DataFrame({"X_cols": X_train.columns})

# ファイル保存（csv）
score_name = "score.csv"
importance_name = "importance.csv"
cols_name = "X_cols.csv"

score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)

score_all.to_csv(score_path, index=False)
importance_all.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)


data/1_output/result_20220924112108
y_weekday tree


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8075,0.8107,0.8343,0.8223,233,57,67,287


y_weekday RandomForest


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.8059,0.7994,0.8398,0.8191,233,57,67,287


y_weekday GradientBoosting


...

,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.7096,0.7249,0.7355,0.7302,233,57,67,287


y_weekend RandomForest


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816
1,test,0.7981,0.7908,0.8288,0.8094,233,57,67,287


y_weekend GradientBoosting


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp
0,train,0.8554,0.8134,0.9034,0.8560,685,0,0,816
1,test,0.8059,0.7794,0.8500,0.8132,233,57,67,287


In [99]:
# 平日モデルの評価

score_all.loc[score_all["model_target"] == "y_weekday"]


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp,model_name,model_target
0,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816,tree,y_weekday
1,test,0.8075,0.8107,0.8343,0.8223,233,57,67,287,tree,y_weekday
2,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816,RandomForest,y_weekday
3,test,0.8059,0.7994,0.8398,0.8191,233,57,67,287,RandomForest,y_weekday
4,train,0.8721,0.8958,0.8723,0.8839,685,0,0,816,GradientBoosting,y_weekday
5,test,0.8276,0.8418,0.8442,0.8430,233,57,67,287,GradientBoosting,y_weekday


In [100]:
# 休日モデルの評価

score_all.loc[score_all["model_target"] == "y_weekend"]


,DataCategory,accuracy,recall,precision,f1,tn,fp,fn,tp,model_name,model_target
6,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816,tree,y_weekend
7,test,0.7096,0.7249,0.7355,0.7302,233,57,67,287,tree,y_weekend
8,train,1.0000,1.0000,1.0000,1.0000,685,0,0,816,RandomForest,y_weekend
9,test,0.7981,0.7908,0.8288,0.8094,233,57,67,287,RandomForest,y_weekend
10,train,0.8554,0.8134,0.9034,0.8560,685,0,0,816,GradientBoosting,y_weekend
11,test,0.8059,0.7794,0.8500,0.8132,233,57,67,287,GradientBoosting,y_weekend


In [101]:
# 重要度の高い変数
# 勾配Boostingモデル（平日）

importance_all.loc[
    (importance_all["model_target"] == "y_weekday") &\
    (importance_all["model_name"] == "GradientBoosting")
].head(20)

,cols,importance,model_name,model_target
426,order_weekend,0.4848,GradientBoosting,y_weekday
427,order_weekday,0.3508,GradientBoosting,y_weekday
428,order,0.0287,GradientBoosting,y_weekday
429,order_time_12,0.0120,GradientBoosting,y_weekday
430,order_time_21,0.0119,GradientBoosting,y_weekday
431,delta_avg,0.0117,GradientBoosting,y_weekday
432,order_time_13,0.0110,GradientBoosting,y_weekday
433,order_cancel,0.0100,GradientBoosting,y_weekday
434,order_time_14,0.0095,GradientBoosting,y_weekday
435,order_time_15,0.0094,GradientBoosting,y_weekday


In [103]:
# 重要度の高い変数
# 勾配Boostingモデル（休日）

importance_all.loc[
    (importance_all["model_target"] == "y_weekend") &
    (importance_all["model_name"] == "GradientBoosting")
].head(20)


,cols,importance,model_name,model_target
1065,order_weekend,0.4295,GradientBoosting,y_weekend
1066,order_weekday,0.4016,GradientBoosting,y_weekend
1067,order_time_11,0.0144,GradientBoosting,y_weekend
1068,order_time_16,0.0129,GradientBoosting,y_weekend
1069,order_time_20,0.0125,GradientBoosting,y_weekend
1070,order_cancel,0.0122,GradientBoosting,y_weekend
1071,delta_avg,0.0108,GradientBoosting,y_weekend
1072,order_time_19,0.0104,GradientBoosting,y_weekend
1073,order_time_17,0.0099,GradientBoosting,y_weekend
1074,order_time_15,0.0090,GradientBoosting,y_weekend
